In [4]:
from __future__ import annotations
from typing import Tuple, List, Dict, KeysView, Iterable
from random import uniform
from gurobipy import Model, tupledict, GurobiError, GRB
from graph_tool import Graph, EdgePropertyMap
from graph_tool.generation import complete_graph
from graph_tool.flow import boykov_kolmogorov_max_flow, min_st_cut #min_st_cut gives the dual sol. for x*
from graph_tool.topology import label_components

In [5]:
Vertex = int
Arc = Tuple[Vertex, Vertex]
Tour = List[Vertex]

class TSPIntance:
    n: int
    x: List[float] #contains x coordinates for all our vertices 
    y : List[float]
    cost: Dict[Arc, float] #contains y coordinates for all our vertices. Vertex i is formed by x[i], y[i]
    

    def __init__(self, x: List[float], y: List[float], costs:List[float]):
        assert len(x) == len(y), "nodes and costs lists must have the same length"

        self.n = len(x)
        self.x = x
        self.y = y
        self.cost = {
            (i, j): 10
            for i in self.vertices()
            for j in self.vertices()
            if i != j
        }
        self.cost[(0,4)]=1
        self.cost[(1,2)]=-1
        self.cost[(2,3)]=-1
        self.cost[(3,1)]=-1
        

    def vertices(self) -> Iterable[Vertex]: #we define this so we can easily compute self.cost
        return range(self.n)
    
    def arcs(self) -> KeysView:
        return self.cost.keys()

    @staticmethod
    def random(n: int) -> TSPIntance:
        x = [uniform(0, 10) for _ in range(n)]
        y = [uniform(0, 10) for _ in range(n)]
        costs= [uniform(-10,10) for _ in range(n*(n-1))]
        return TSPIntance(x=x, y=y, costs=costs)

In [6]:
class TSPSolution:
    tour: Tour
    cost: float

    def __init__(self, tour: Tour, **kwargs):
        assert 'cost' in kwargs or 'instance' in kwargs, \
            "You must pass the tour cost or a TSP instance to compute it"

        if 'cost' in kwargs:
            self.cost = kwargs.get('cost')
        elif 'instance' in kwargs:
            tsp = kwargs.get('instance')
            self.cost = sum(
                tsp.cost[i, j]
                for i in tour[:-1]
                for j in tour[1:]
            )

        self.tour = tour

    def __str__(self) -> str:
        return "[" + ', '.join(map(str, self.tour)) + f"] - Cost: {self.cost:.2f}"

In [7]:
class BranchAndCutFractionalSolver:
    tsp: TSPIntance
    m: Model
    x: tupledict
    G: Graph
    capacity: EdgePropertyMap

    def __init__(self, tsp: TSPIntance):
        self.tsp = tsp
        self.m = Model()
        self.x = self.m.addVars(self.tsp.arcs(), obj=self.tsp.cost, vtype=GRB.BINARY, name='x') 
        self.__build_model()
        self.__build_graph()

    def __build_model(self) -> None:
        self.m.addConstrs(self.x.sum(i, '*') == 1 for i in self.tsp.vertices())
        self.m.addConstrs(self.x.sum('*', i) == 1 for i in self.tsp.vertices())
    
    def __build_graph(self) -> None: #wE ALWAYS CREATE MAPS THIS WAY. WE JUST COPY PASTE
        self.G = complete_graph(N=self.tsp.n, self_loops=False, directed=True) #creates graph with arcs between every pair of nodes
        self.capacity = self.G.new_edge_property(value_type='double')
        self.G.edge_properties['capacity'] = self.capacity #we add a key to our dictionary with the capacity
    
    def solve(self) -> TSPSolution:
        self.m.setParam(GRB.Param.LazyConstraints, 1)
        self.m.optimize(lambda _, where: self.__separate(where=where)) #same as integer 

        if self.m.Status != GRB.OPTIMAL:
            raise RuntimeError("Could not solve TSP model to optimality")
        
        return TSPSolution(tour=self.__tour_starting_at(0), cost=self.m.ObjVal)
    
    def __separate(self, where: int) -> None: #we define the separation procedure

        #We only want to separate when we are in a MIPNODE optimal, or a MIPSOL
        #MIPNODE is a state in which we are checking for a sol in the tree.
        #MIPSOL means gurobi found a possibly feasible solution.
        if where not in (GRB.Callback.MIPSOL, GRB.Callback.MIPNODE):
            return        
        # In MIPNODE, we must ensure that we have the optimal solution to the
        # linear relaxation at that node. Otherwise we are not "authorised"
        # to call cbGetNodeRel. See:
        # https://www.gurobi.com/documentation/9.1/refman/py_model_cbgetnoderel.html

        #We check if we are in MIPNODE but not yet found optimal
        if where == GRB.Callback.MIPNODE and self.m.cbGet(GRB.Callback.MIPNODE_STATUS) != GRB.OPTIMAL:
            return         
        
        self.__update_capacity() #updates all capacities to their Xij value. 

        source = self.G.vertex(0) #this is our way to get the vertex at position 0.
        remaining= set(range(1,self.tsp.n-1)) 
        #already_added = {source} #here we keep track of nodes we have not already added.
        residual = boykov_kolmogorov_max_flow(
                g=self.G, source=source, target=self.tsp.n-1,
                capacity=self.capacity
            )
        residual_components = self.get_connected_components(residual)

        for component in residual_components:
        # Skip trivial components with only one node
            if len(component) < 2:
                continue

            # Calculate total flow within the component
            total_flow = 0
            for i in component:
                for j in component:
                    if i != j:
                        edge = residual.edge(i, j)
                        if edge is not None:
                            total_flow += self.capacity[edge]

            # Check SEC violation: total flow should not exceed |S| - 1
            if total_flow > len(component) - 1:
                self.__add_sec_for(component) 
                
                

        
    def __update_capacity(self) -> None:
        for e in self.G.edges():
            i, j = e.source(), e.target()

            try: # If we are in MIPSOL
                xval = self.m.cbGetSolution(self.x[i,j])
            except: # If we are in MIPNODE
                xval = self.m.cbGetNodeRel(self.x[i,j]) 

            self.capacity[e] = xval

    
    def get_connected_components(residual_graph):
        # Identify the connected components
        component_labels, _ = label_components(residual_graph)
        
        # Group vertices by component label
        components = {}
        for v in residual_graph.vertices():
            label = component_labels[v]
            if label not in components:
                components[label] = []
            components[label].append(int(v))  # Convert vertex to index
        
        # Return the list of components, where each component is a list of vertices
        return list(components.values())

        
    def __add_sec_for(self, subtour: Tour) -> None:
        print("Adding subtour for [" + ', '.join(map(str, subtour)) + "]")
        self.m.cbLazy(
            sum(
                self.x[i, j]
                for i, j in self.tsp.arcs()
                if i in subtour and j not in subtour
            ) >= 1
        )

In [9]:
tsp = TSPIntance.random(n=5)
solver = BranchAndCutFractionalSolver(tsp=tsp)
solution = solver.solve()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-12-02
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
LazyConstraints  1

Optimize a model with 10 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x3d17ee98
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]


TypeError: BranchAndCutFractionalSolver.get_connected_components() takes 1 positional argument but 2 were given

Exception ignored in: 'gurobipy._core.callbackstub'
Traceback (most recent call last):
  File "src/gurobipy/callback.pxi", line 101, in gurobipy._core.CallbackClass.callback
  File "/tmp/ipykernel_567485/3538379101.py", line 26, in <lambda>
  File "/tmp/ipykernel_567485/3538379101.py", line 58, in __separate
TypeError: BranchAndCutFractionalSolver.get_connected_components() takes 1 positional argument but 2 were given


Found heuristic solution: objective 50.0000000
Presolve time: 0.03s
Presolved: 10 rows, 20 columns, 40 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)

Root relaxation: objective 8.000000e+00, 2 iterations, 0.01 seconds (0.00 work units)


TypeError: BranchAndCutFractionalSolver.get_connected_components() takes 1 positional argument but 2 were given

Exception ignored in: 'gurobipy._core.callbackstub'
Traceback (most recent call last):
  File "src/gurobipy/callback.pxi", line 101, in gurobipy._core.CallbackClass.callback
  File "/tmp/ipykernel_567485/3538379101.py", line 26, in <lambda>
  File "/tmp/ipykernel_567485/3538379101.py", line 58, in __separate
TypeError: BranchAndCutFractionalSolver.get_connected_components() takes 1 positional argument but 2 were given



    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       8.0000000    8.00000  0.00%     -    4s

Explored 1 nodes (2 simplex iterations) in 4.09 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 8 50 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e+00, best bound 8.000000000000e+00, gap 0.0000%

User-callback calls 114, time in user-callback 3.99 sec


AttributeError: 'BranchAndCutFractionalSolver' object has no attribute '_BranchAndCutFractionalSolver__tour_starting_at'

In [ ]:
print(solution.tour)

[0, 13, 9, 7, 10, 17, 19, 1, 12, 4, 2, 15, 6, 16, 3, 8, 18, 11, 14, 5]
